# Imaging Methods in Granular & Complex Fluids
# Day 3: Particle finding 2

Karen Daniels, Distinguished Professor of Physics, NC State University kdaniel@ncsu.edu

Visiting Fulbright-Nehru Scholar,  IISc Civil Engineering, September 2023 to January 2024

## Learning objectives

* Use `scikit-image` to find particle centroids using a convolution method
* Evaluate subpixel resolution of particle centroids


# Pre-class

* *Pre-class reading:* Section 2.4.1 of Shattuck, M. D. Chapter 2: Experimental Techniques. Handbook of Granular Materials. CRC Press (2016)  (PDF is saved in GDrive)
* *Bring to class:* laptop with anaconda with `scipy`, `matplotlib`, and `scikit-image` installed, photos of particles (circular or otherwise)

# In-Class

Load the libraries we'll need for today

In [ ]:
import numpy as np              # for scientific calculations
import matplotlib.pyplot as plt # for plotting
import skimage                  # for image-processing

from skimage.util import img_as_ubyte
from scipy import ndimage
from skimage.feature import peak_local_max
from matplotlib.patches import Circle
from skimage.segmentation import watershed

In [ ]:
# if you like interactive plots, run these
# otherwise don't

%matplotlib notebook
plt.ion()

## Using kernel convolution to find circle-like objects

We're going to use a kernel (template of the expected particle image) to locate objects in the image that match the kernel. 

$$ {\tilde I} (x,y) = k * I =  \sum_{i=0}^{L-1}  \sum_{j=0}^{L-1} k(i,j)  I(x-i, y-j)$$

$$ k*I = \mathrm{fft}^{-1} [ \mathrm{fft}(k) \, \mathrm{fft}(I) ] $$

Feel free to use the image below, or to substitute your own. 

The `PEdisks.png` image is solid white circles on a black background, so if your image is the other way around, there's a line of code you can uncomment which will invert the image.

In [ ]:
img = skimage.io.imread('PEdisks.png')
#img = io.imread('subdir/my-picture.jpg')   #use this if your picture is in a subdirectory (Mac/Linux)
#img = io.imread('subdir\my-picture.jpg')   #use this if your picture is in a subdirectory (Windows)

#we'll be doing floating point operations, so convert from integer to float
image = img_as_ubyte(img[:,:,2]).astype(float)  

#uncomment this line if you want invert black-to-white
#image = -image

#prepare image to have zero mean and max = 1
image = image - np.mean(image)
image = image/np.max(image)

plt.imshow(image,cmap=plt.cm.gray)
plt.colorbar()
plt.show()

**Based on what you know about convolutions (either ingetrals or sums), why is it important for the data to have zero mean? Why is it nice for it to have a max value of 1?**

*... your thoughts here ...*

### Making a kernel for convolutions

Using the interactive mode you should see that the two disk sizes have radii of about 20 and 30 pixels. The code below makes a kernel which is either a solid disk, or a circular perimeter (ring). We'll try both templates to see which works better. 

In [ ]:
# make a filled circular kernel with a known radius and a little padding at the outside 
# R does not neet to be an integer

def disckernel(R):
    kdim = int(np.ceil(2*(float(R)+0.5)))  # +0.5 gives at least a blank pixel around the outside edge
    c = (kdim-1)/2.0  # +/- pixels from center to edge of kernal
    x, y = np.indices((kdim, kdim))
    kernel = (x-c)**2 + (y-c)**2 < R**2
    return(kernel - np.mean(kernel), c)   # return kernal with zero mean, and +/- size of kernel

# make a circular kernel with a known radius and width w, with a little padding at the outside 
# R does not neet to be an integer
def circkernel(R, w):
    kdim = int(np.ceil(2*(float(R)+0.5)))  # +0.5 gives at least a blank pixel around the outside edge
    c = (kdim-1)/2.0  # +/- pixels from center to edge of kernal
    kernel = np.zeros((kdim, kdim))
    x, y = np.indices((kdim, kdim))
    kernel = ((x-c)**2 + (y-c)**2 < R**2 ) &  ((x-c)**2 + (y-c)**2 > (R-w)**2 )
    return(kernel - np.mean(kernel), c)   # return kernal with zero mean, and +/- size of kernel


R = 19.5

f, (ax0, ax1) = plt.subplots(1,2, figsize=(6,3))
dkern, dx = disckernel(R)
ax0.imshow(dkern, extent=[-dx,dx,-dx,dx],cmap=plt.cm.gray)
ax0.set_title('Disc with R=' + str(R))

w = 1.5
ckern, cx = circkernel(R, 1.5)
ax1.imshow(ckern, extent=[-cx,cx,-cx,cx],cmap=plt.cm.gray)
ax1.set_title('Circle with R=' + str(R) + ' w=' + str(w))

plt.show()

### Find the large circles using the disk kernel

We're going to do this using the `scipy` convolution package, but if you know Fourier transforms, doing your convolutions in Fourier space would be faster than this.

$$ {\tilde I} (x,y) = k * I =  \sum_{i=0}^{L-1}  \sum_{j=0}^{L-1} k(i,j)  I(x-i, y-j)$$

$$ k*I = \mathrm{fft}^{-1} [ \mathrm{fft}(k) \, \mathrm{fft}(I) ] $$

In [ ]:
# do the convolution for just one radius 
R = 19.0
dkern, dx = disckernel(R)
conv = ndimage.convolve(image, dkern, mode='constant', cval=0.0)
conv = conv/np.max(conv)

# each local maximum in the convolution is the center of a circle
# setting min_distance discards peaks that are too close together
max = peak_local_max(conv, min_distance=int(np.ceil(R)))

# plot all circles found
f, (ax0,ax1) = plt.subplots(1,2, figsize=(8,4))
ax0.imshow(conv)
ax0.set_title('Convolution with R=' + str(R))
#ax0.colormap()

ax1.imshow(image, cmap=plt.cm.gray)
for center_y, center_x in max:
   circle = Circle((center_x, center_y), R, edgecolor='magenta', facecolor="none", linewidth=2, alpha=0.5)
   ax1.add_patch(circle) 
ax1.set_title('Finding disks with R=' + str(R))

plt.show()

Modify the radius to +/- a few pixels, and also try the smaller radius particles. You should notice that (like the Hough), the convolution detects particles of the wrong size, but that score is better/worse when there's a good match. 

Therefore, we will switch to detecting circular rings rather than solid disks.

In [ ]:
img = skimage.io.imread('PEdisks.png')
image = img_as_ubyte(img[:,:,2]).astype(float)
erodedimage = skimage.morphology.erosion(image)

# the difference between the original and eroded image
# this gives a wider "rim" than Canny edge detection, and often works better
image = image - erodedimage 

# set zero mean and max=1
image = image - np.mean(image)
image = image/np.max(image)
plt.imshow(image, cmap=plt.cm.gray)
plt.colorbar()
plt.show()

In [ ]:
# do the convolution for just one radius 
R = 19.5
w = 1.5
ckern, cx = circkernel(R, w)
conv = ndimage.convolve(image, ckern, mode='constant', cval=0.0)
conv = conv/np.max(conv)

# each local maximum in the convolution is the center of a circle
# setting min_distance discards peaks that are too close together
max = peak_local_max(conv, min_distance=int(np.ceil(R)))

# plot all circles found
f, (ax0,ax1) = plt.subplots(1,2, figsize=(8,4))
ax0.imshow(conv)
ax0.set_title('Convolution with R=' + str(R))
#ax0.colormap()

ax1.imshow(image, cmap=plt.cm.gray)
for center_y, center_x in max:
   circle = Circle((center_x, center_y), R, edgecolor='magenta', facecolor="none", linewidth=2, alpha=0.5)
   ax1.add_patch(circle) 
ax1.set_title('Finding disks with R=' + str(R))

plt.show()

**Try a few different values of $R, w$ and see if you can decide which settings give the strongest, clearest peaks.** You should see that the scores for large particles (for small $R$, and vice versa) are *much* weaker than when using the solid disks. What shapes do the "mistakes" in small vs. larger look like in the convolution matrix?

*... your thoughts here ...*

### Write a function to do the convolutions and show them

This will make things simpler going forward. You can turn the display feature on and off, depending on whether or not you're in debugging mode.

In [ ]:
def convcirc(image, R, w):
    ckern, x = circkernel(R, w)
    conv = ndimage.convolve(image, ckern, mode='constant', cval=0.0)
    conv = conv/np.max(conv)
    max = peak_local_max(conv, min_distance=int(np.ceil(R)))
    
    if 0:  # toggle 0/1 depending on whether you want to show the results
        f, ax0 = plt.subplots()
        ax0.imshow(conv, cmap=plt.cm.gray)
        for center_y, center_x in max:
           ax0.plot(center_x, center_y, 'rx')
           circle = Circle((center_x, center_y), R, edgecolor='red', facecolor="none", linewidth=2, alpha=0.5)
           ax0.add_patch(circle)
        plt.title('R = ' + str(R)  + ' convolution')
        plt.show()
    return(max)

In [ ]:
w=1.5
Rs = 19.5
maxS = convcirc(image, Rs, w)
Rl = 29.0
maxL = convcirc(image, Rl, w)

f, ax0 = plt.subplots()#ax1.imshow(convS, cmap=plt.cm.gray)
ax0.imshow(image, cmap=plt.cm.gray)
for center_y, center_x in maxS:
   circle = Circle((center_x, center_y), Rs, edgecolor='blue', facecolor="none", linewidth=2, alpha=0.5)
   ax0.add_patch(circle)
for center_y, center_x in maxL:
   circle = Circle((center_x, center_y), Rl, edgecolor='red', facecolor="none", linewidth=2, alpha=0.5)
   ax0.add_patch(circle)  
plt.title('blue = small, red = large')
plt.show()

We're still getting false-postives for smalls-inside-larges, so let's first optimize the choice of $R$ to make this less of a problem. We'll do this by varying $R$ and looking at how many circles are detected.

In [ ]:
# a range of R values to try (covers both smalls and larges)
Rvals = np.arange(16., 34., 0.2)
numfound = np.zeros(Rvals.size)
w = 1.5

for i in range(Rvals.size):
    max = convcirc(image, Rvals[i], w)
    numfound[i] = max.size
    
plt.plot(Rvals, numfound, 'x')
plt.xlabel('kernel radius [pixels]')
plt.ylabel('number found')
plt.show()

You should see 2 plateaus, one for each particle size. For best restuls, pick a smallish value for each $R$ (such that it doesn't over-detect much). This will give tall, wide peaks in the convolution matrix, and this will make it easiest to pick the correct centroids.

Next, we can screen out the weaker peaks by looking at a histogram of the values at the peaks.

In [ ]:
R = 20.0
w = 1.5
ckern, x = circkernel(R, w)
conv = ndimage.convolve(image, ckern, mode='constant', cval=0.0)
conv = conv/np.max(conv)
max = peak_local_max(conv, min_distance=int(np.ceil(R)))
peak = np.zeros(max.shape[0])
for i in range(max.shape[0]):
    peak[i] = conv[max[i,0], max[i,1]]

plt.hist(peak)
plt.xlabel('peak height')
plt.ylabel('number of occurances')
plt.show()

From this plot, it looks safe to discard all peaks less than 0.55. So, we'll revise our particle-finding function, and test this out.

In [ ]:
def convcirc(image, R, w, thresh):
    ckern, x = circkernel(R, w)
    conv = ndimage.convolve(image, ckern, mode='constant', cval=0.0)
    conv = conv/np.max(conv)
    max = peak_local_max(conv, min_distance=int(np.ceil(R)), threshold_abs = thresh)
    
    if 0:  # toggle 0/1 depending on whether you want to show the results
        f, ax0 = plt.subplots()
        ax0.imshow(conv, cmap=plt.cm.gray)
        for center_y, center_x in max:
           ax0.plot(center_x, center_y, 'rx')
           circle = Circle((center_x, center_y), R, edgecolor='red', facecolor="none", linewidth=2, alpha=0.5)
           ax0.add_patch(circle)
        plt.title('R = ' + str(R)  + ' convolution')
        plt.show()
    return(max)

max = convcirc(image, 19.5, 1.5, 0.55)
print('Found ' + str(max.shape[0]) + ' particles')

In [ ]:
w=1.5
Rs = 19.5
maxS = convcirc(image, Rs, w, 0.55)
Rl = 29.0
maxL = convcirc(image, Rl, w, 0.55)

f, ax0 = plt.subplots()#ax1.imshow(convS, cmap=plt.cm.gray)
ax0.imshow(image, cmap=plt.cm.gray)
for center_y, center_x in maxS:
   circle = Circle((center_x, center_y), Rs, edgecolor='blue', facecolor="none", linewidth=2, alpha=0.5)
   ax0.add_patch(circle)
for center_y, center_x in maxL:
   circle = Circle((center_x, center_y), Rl, edgecolor='red', facecolor="none", linewidth=2, alpha=0.5)
   ax0.add_patch(circle)  
plt.title('blue = small, red = large')
plt.show()

If all has gone well .... this is a perfect result, one circle for each particle, sorted into large and small! 

**A few things to think about:**
* can you see how to fix up the Hough detection (from Day 2) to perform a similar screening?
* how would you modify the code to find particles of a large variety of radii?

*... your thoughts here ...*

## Accuracy vs. precision

At the start of this process, we were getting *inaccurate* results: detecting the wrong size circle in a wrong place. This is the most serious type of error to get rid of, and we succeeded in that goal. 

Next, we want to evaluate the *precision* of our measurements: what is the number of significant digits in the list of centroids. Sadly, the `peak_local_max()` function returns only integer positions, so we'll first improve that precision, then then examine histograms of the *remainder* of the (decimal) centroid coordinates. If these are strongly peaked at 0 and 1, then we have only $\pm 0.5$ pixel precision. However, convolution methods can usually achieve sub-pixel precision (much better than Hough).

In [ ]:
# revising our function one more time

def convcirc(image, R, w, thresh):
    ckern, x = circkernel(R, w)
    conv = ndimage.convolve(image, ckern, mode='constant', cval=0.0)
    conv = conv/np.max(conv)
    max = peak_local_max(conv, min_distance=int(np.ceil(R)), threshold_abs = thresh)
    # these are just integer values
    #print(max) # you can comment this line out once debugging is done
    
    fitmax = np.copy(max.astype(float))
    p = 2 # we'll do a weighted average of this number of pixels on each side
    prange = np.arange(-p,p+1) # the +1 is since index  goes *up to* the second value
    for i in range(max.shape[0]):
        x0 = max[i,0]
        y0 = max[i,1]
        fitmax[i,0] = np.sum(np.dot(x0+prange, conv[x0-p:x0+p+1, y0]))/np.sum(conv[x0-p:x0+p+1, y0])
        fitmax[i,1] = np.sum(np.dot(y0+prange, conv[x0, y0-p:y0+p+1]))/np.sum(conv[x0, y0-p:y0+p+1])
    return (fitmax)

fitmaxS = convcirc(image, 19.5, 1.5, 0.55)
fitmaxL = convcirc(image, 29.0, 1.5, 0.55)

#put all coordinates into a single 1D array
allmax = np.reshape(np.concatenate([fitmaxS, fitmaxL]), fitmaxL.size + fitmaxS.size)

#calculate their remainders
remainders = allmax - np.floor(allmax)

plt.hist(remainders)
plt.xlabel('Remainder')
plt.ylabel('# of occurances')
plt.show()

You'll likey see that you only have a precision of about $\pm 0.5$ pixels. Feel free to mess around and see if you can do better! Better choice of radius? Better ring method? Better peak-fitting (instead of weigthed average)? Better kernel (smoother edges?)

Eric Weeks' webpage has examples of better/worse subpixel resolution: https://physics.emory.edu/faculty/weeks/idl/tracking.html 

In [ ]:
# your space to try and do better ... 

 ## Watershed segmentation
 
This is a quick demonstration of watershed segmentation, which `skimage` provides as a built-in package. The code here largely follows the example given in the [documentation](https://scikit-image.org/docs/stable/auto_examples/segmentation/index.html)

In [ ]:
# Generate an initial image with two overlapping circles
x, y = np.indices((150, 150))
x1, y1, x2, y2 = 58, 52, 74, 100
r1, r2 = 25, 40
mask_circle1 = (x - x1)**2 + (y - y1)**2 < r1**2
mask_circle2 = (x - x2)**2 + (y - y2)**2 < r2**2
image = np.logical_or(mask_circle1, mask_circle2)

plt.imshow(image, cmap=plt.cm.gray)
plt.title('Overlapping objects')
plt.show

In [ ]:
# Now we want to separate the two objects in image
# Generate the markers as local maxima of the distance to the background (zero)
distance = ndimage.distance_transform_edt(image)
coords = peak_local_max(distance, footprint=np.ones((3, 3)), labels=image)
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = ndimage.label(mask)
labels = watershed(-distance, markers, mask=image)

fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Overlapping objects')
ax[1].imshow(-distance, cmap=plt.cm.gray)
ax[1].set_title('Distances')
ax[2].imshow(labels, cmap=plt.cm.nipy_spectral)
ax[2].set_title('Separated objects')

for a in ax:
    a.set_axis_off()

fig.tight_layout()
plt.show()

In [ ]:
# now, pick and image from your collection, and segement the objects 

img = skimage.io.imread('apples-oranges.jpg')
image = img_as_ubyte(img[:,:,2])  # just the blue channel, which is similar for both fruits

img = skimage.io.imread('PEdisks.png')
image = img_as_ubyte(img)

plt.imshow(image, cmap=plt.cm.gray)
plt.show()

# Post-class

## Every week

If you and your group worked together on a single notebook, don't forget to email each other a copy of the final version.

Please feel free to complete any parts you didn't finish, and ask questions at the start of the next class.